In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [40]:
df=pd.read_csv(r"..\csv_files\titanic\train.csv")
test=pd.read_csv(r"..\csv_files\titanic\test.csv")

In [41]:
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Cabin'] = df['Cabin'].notnull().astype(int)  # Convert to binary feature
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

In [42]:
test['Age'].fillna(test['Age'].median(), inplace=True)
test['Cabin'] = test['Cabin'].notnull().astype(int)  # Convert to binary feature
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace=True)

In [43]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1


In [44]:
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1


In [45]:
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
df['Title'] = df['Title'].replace(['Mme'], 'Mrs')
df['Title'] = df['Title'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer')
df['Title'] = df['Title'].replace(['Don', 'Dona', 'Lady', 'Sir', 'Countess', 'Jonkheer'], 'Royalty')


<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\hp\AppData\Local\Temp\ipykernel_6140\1450961300.py:1: SyntaxWarning: invalid escape sequence '\.'
  df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


In [46]:
test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test['Title'] = test['Title'].replace(['Mlle', 'Ms'], 'Miss')
test['Title'] = test['Title'].replace(['Mme'], 'Mrs')
test['Title'] = test['Title'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer')
test['Title'] = test['Title'].replace(['Don', 'Dona', 'Lady', 'Sir', 'Countess', 'Jonkheer'], 'Royalty')


<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\hp\AppData\Local\Temp\ipykernel_6140\2154131037.py:1: SyntaxWarning: invalid escape sequence '\.'
  test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


In [47]:
df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)


In [48]:
Id = pd.DataFrame({'PassengerId': test['PassengerId']})
test.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)


In [49]:
df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'Title'], drop_first=True)


In [50]:
test = pd.get_dummies(test, columns=['Sex', 'Embarked', 'Title'], drop_first=True)


In [51]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['Age', 'Fare', 'FamilySize']] = scaler.fit_transform(df[['Age', 'Fare', 'FamilySize']])
test[['Age', 'Fare', 'FamilySize']] = scaler.fit_transform(test[['Age', 'Fare', 'FamilySize']])


In [52]:
X = df.drop("Survived", axis=1)
y = df["Survived"]

In [53]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)
y_pred = model.predict(test)
from sklearn.metrics import accuracy_score

In [54]:
import xgboost as xgb
model = xgb.XGBClassifier(random_state=42)
model.fit(X, y)
y_pred = model.predict(test)


In [ ]:
submission = pd.DataFrame({
    'PassengerId': Id['PassengerId'],
    'Survived': y_pred
    })
submission.to_csv('submission.csv', index=False)

KeyError: 'PassengerId'